# ehiniumChat | Notebook Submission

این نوت بوک برای تمرین کلاسی ساخته شده تا پروژه قابل اجرا و قابل بررسی باشد.

فایل های اصلی پروژه:
- `app.py` سرور چت (aiohttp + WebSocket + SQLite)
- `admin_cli.py` ابزار مدیریت کاربران و گروه ها
- `README.md` توضیحات نصب و اجرا

نکته: پروژه اصلی برای لینوکس و مسیر `/opt/ehiniumChat` نوشته شده. برای اینکه همین نوت بوک روی ویندوز هم بدون دستکاری فایل ها اجرا شود، در سلول های تست، مسیرها را به یک پوشه موقت تغییر میدهیم.


## 1) خلاصه پروژه

این توضیح از README گرفته شده است:

In [ ]:
print('# ehiniumChat\n\n**ehiniumChat** is a lightweight, private group chat system designed for Linux servers. \nIt uses only system packages, stores data in SQLite, and provides a clean Telegram-like UI with realtime messaging.\n\nThis project is intentionally simple, auditable, and self-hosted.\n\n## Features\n\n- Private login (no public signup)\n- User-defined groups and memberships\n- Realtime messaging via WebSocket\n- Message pagination (loads latest 10, loads older on scroll)\n- Unread message dots and counters\n- Group info panel (members + last seen)\n- File upload with image preview\n- Mobile responsive UI\n- SQLite database (single file)\n- No external services, no pip, no npm\n\n## Requirements\n\n- **Linux server (Ubuntu recommended, any version)**\n- Root or sudo access\n- Ability to install packages using `apt`\n- Optional: domain + CDN (for HTTPS on client side)\n\nUsed system packages only:\n- `python3`\n- `python3-aiohttp`\n\n## Important note for Iran access servers\n\nIf your server is **Iran-access (زمان قطعی اینترنت)**:\n\n### 1) DNS resolution\nPublic resolvers like **8.8.8.8** or **1.1.1.1** usually do **not work**.\n\nMake sure your server:\n- Uses datacenter-provided DNS, or\n- Uses Iranian DNS resolvers\n\n### 2) Use Iranian apt mirrors\nDefault Ubuntu mirrors may fail.  \nRecommended tools to switch mirrors automatically:\n\nhttps://github.com/mexenon/syshelper  \nhttps://github.com/GeeDook/mirava\n\nAfter fixing mirrors:\n```bash\nsudo apt update\n```\n\n### Download project without GitHub access\nIf GitHub is blocked, download the packaged file from this mirror:  \n```bash\nhttps://uploadkon.ir/uploads/3cfb13_26ehiniumChat-tar.gz\n```\n')

## 2) آماده سازی محیط

In [ ]:
import sys, subprocess, pkgutil

def ensure(pkg):
    if pkgutil.find_loader(pkg) is None:
        print('Installing', pkg)
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
    else:
        print(pkg, 'OK')

# aiohttp برای اجرای واقعی سرور لازم است، ولی برای تست های این نوت بوک ضروری نیست.
ensure('aiohttp')


## 3) بارگذاری ماژول `app.py` بدون تغییر فایل

در این بخش `app.py` را مثل یک ماژول پایتون لود میکنیم تا بتوانیم توابعش را تست کنیم.

In [ ]:
import importlib.util, pathlib, tempfile, os, time, secrets, sqlite3

APP_FILE = pathlib.Path("app.py").resolve()
print("Using app.py at:", APP_FILE)

spec = importlib.util.spec_from_file_location("ehinium_app", str(APP_FILE))
ehc = importlib.util.module_from_spec(spec)
spec.loader.exec_module(ehc)

print("Loaded module:", ehc.APP_NAME)
print("Default BASE_DIR:", ehc.BASE_DIR)


## 4) ساخت یک محیط تست محلی (بدون `/opt`)

در پروژه اصلی مسیرها ثابت هستند. اینجا برای اجرای روی ویندوز/لوکال، آنها را به یک پوشه موقت تغییر میدهیم و یک دیتابیس تازه میسازیم.

In [ ]:
import pathlib, tempfile, os

tmp = tempfile.TemporaryDirectory()
BASE = pathlib.Path(tmp.name) / "ehiniumChat"
BASE.mkdir(parents=True, exist_ok=True)

# Patch paths inside the loaded module
ehc.BASE_DIR = str(BASE)
ehc.DB_PATH = str(BASE / "ehiniumchat.db")
ehc.SECRET_PATH = str(BASE / "secret.key")
ehc.UPLOAD_DIR = str(BASE / "uploads")

print("Patched BASE_DIR:", ehc.BASE_DIR)
print("Patched DB_PATH:", ehc.DB_PATH)
print("Patched SECRET_PATH:", ehc.SECRET_PATH)
print("Patched UPLOAD_DIR:", ehc.UPLOAD_DIR)

pathlib.Path(ehc.UPLOAD_DIR).mkdir(parents=True, exist_ok=True)


## 5) ایجاد secret.key و ساخت دیتابیس

طبق README، `secret.key` برای امضای کوکی ها لازم است. سپس `init_db()` جداول SQLite را میسازد.

In [ ]:
import secrets, pathlib

# create secret key
secret_hex = secrets.token_hex(32)
pathlib.Path(ehc.SECRET_PATH).write_text(secret_hex, encoding="utf-8")

# load secret into module global
ehc.SECRET = ehc.read_secret()

# init db
ehc.init_db()

print("secret.key created (len):", len(secret_hex))
print("DB exists:", pathlib.Path(ehc.DB_PATH).exists())


## 6) تست امضا و اعتبارسنجی کوکی

این تست نشان میدهد مکانیزم لاگین از نظر امضای HMAC کار میکند.

In [ ]:
import time

uid = 123
exp = int(time.time()) + 3600
cookie = ehc.sign_cookie(uid, exp)
print("cookie:", cookie[:40] + "...")

verified = ehc.verify_cookie(cookie)
print("verified user_id:", verified)

assert verified == uid


## 7) ساخت یک کاربر نمونه و تست login در دیتابیس

برای تست ساده، یک کاربر داخل جدول `users` اضافه میکنیم و بررسی میکنیم query ها درست است.

In [ ]:
import sqlite3, time

conn = sqlite3.connect(ehc.DB_PATH)
cur = conn.cursor()
cur.execute("INSERT INTO users (username, display_name, password, is_admin, last_seen) VALUES (?,?,?,?,?)",
            ("student", "Student", "student123", 0, int(time.time())))
conn.commit()

cur.execute("SELECT id, username, display_name, is_admin FROM users WHERE username=?", ("student",))
row = cur.fetchone()
print("Inserted user:", row)

conn.close()
assert row[1] == "student"


## 8) نکات اجرای پروژه روی سرور لینوکس

1) نصب وابستگی ها با apt
2) کپی فایل ها به `/opt/ehiniumChat`
3) ساخت `secret.key`
4) اجرای `python3 /opt/ehiniumChat/app.py`

دستورهای کامل در README موجود است.

## 9) اشاره به ابزار مدیریت (admin_cli.py)

`admin_cli.py` برای لیست کاربران/گروه ها و مدیریت عضویت هاست. این فایل به صورت پیش فرض دیتابیس را در `/opt/ehiniumChat/ehiniumchat.db` میخواند. پس روی ویندوز فقط برای مطالعه مناسب است، اما روی سرور لینوکس همان طور که README گفته قابل استفاده است.

In [ ]:
# نمایش help ابزار ادمین (بدون اجرا روی DB)
import pathlib, textwrap

cli_text = pathlib.Path("admin_cli.py").read_text(encoding="utf-8").splitlines()
# show the usage() section only
start = next(i for i,l in enumerate(cli_text) if l.strip().startswith("def usage"))
snippet = "\n".join(cli_text[start:start+60])
print(snippet)


## 10) جمع بندی

این نوت بوک:
- پروژه را معرفی میکند
- بدون تغییر فایل های اصلی، ماژول را لود میکند
- یک محیط تست محلی میسازد تا روی ویندوز هم اجرا شود
- چند تست سبک (DB + cookie) انجام میدهد